## Import library

In [ ]:
from tqdm.notebook import trange
import numpy as np
import time

## Configure the device

Device: ChipWhisperer-Lite

In [ ]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEARM'
CRYPTO_TARGET='TINYAES128C' 
SS_VER='SS_VER_2_1'

In [ ]:
import chipwhisperer as cw

In [ ]:
%run "../../Setup_Scripts/Setup_Generic.ipynb"

In [ ]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET" "$SS_VER"
cd ../../../hardware/victims/firmware/simpleserial-aes
make PLATFORM=$1 CRYPTO_TARGET=$2 SS_VER=$3 -j

In [ ]:
cw.program_target(scope, prog, "../../../hardware/victims/firmware/simpleserial-aes/simpleserial-aes-{}.hex".format(PLATFORM))

## Use our collected trace

If you do not have a ChipWhisperer device, you can use the traces we collected and skip the following trace collecting part. It may take some time to load the trace.

Remember to replace the file path.

In [ ]:
your_path = "add_your_path_here"
trace0 = np.loadtxt(your_path+"/data/trace0.txt",delimiter=',')
trace1 = np.loadtxt(your_path+"/data/trace1.txt",delimiter=',')
trace2 = np.loadtxt(your_path+"/data/trace2.txt",delimiter=',')
trace3 = np.loadtxt(your_path+"/data/trace3.txt",delimiter=',')
trace_mix = np.loadtxt(your_path+"/data/trace_mix.txt",delimiter=',')

## Collecting Data

In [ ]:
# generate keys and plaintexts
ktp = cw.ktp.Basic()
trace_array = []
textin_array = []
text0 = []
text1 = []
text2 = []
text3 = []

n_iter = 4000

for i in range(n_iter):
    key, text_t = ktp.next()
    text0.append(text_t)
    key, text_t = ktp.next()
    text1.append(text_t)
    key, text_t = ktp.next()
    text2.append(text_t)
    key, text_t = ktp.next()
    text3.append(text_t)

In [ ]:
# capture interleaved AES
def capture4(key, t0, t1, t2, t3):
    target.set_key(key)
    
    target.set_pt(0, t0)
    target.set_pt(1, t1)
    target.set_pt(2, t2)
    target.set_pt(3, t3)
    
    scope.arm()
    
    target.simpleserial_write('g', t0)
    
    ret = scope.capture()
    if ret:
        print("Target timed out!")
    
    response = target.simpleserial_read('r', 16)
    
    return scope.get_last_trace()

In [ ]:
# capture seperated AES
def capture(key, text):
    target.set_key(key)
    
    scope.arm()
    
    target.simpleserial_write('p', text)
    
    ret = scope.capture()
    if ret:
        print("Target timed out!")
    
    response = target.simpleserial_read('r', 16)
    
    return scope.get_last_trace()

In [ ]:
# capture seperated trace
trace0 = []
trace1 = []
trace2 = []
trace3 = []

scope.adc.samples = 3600

target.reset_comms()

scope.adc.decimate = 8

for i in trange(n_iter):
    trace0.append(np.array(capture(key, text0[i])))
    trace1.append(np.array(capture(key, text1[i])))
    trace2.append(np.array(capture(key, text2[i])))
    trace3.append(np.array(capture(key, text3[i])))

In [ ]:
# capture interleaved trace
trace_mix = []

scope.adc.samples = 16000

target.reset_comms()


scope.adc.decimate = 8

for i in trange(n_iter):
    trace_mix.append(np.array(capture4(key, text1[i], text0[i], text2[i], text3[i])))

# Data process

In [ ]:
from tqdm.notebook import trange,tqdm
import math

def mean(X):
    return np.sum(X, axis=0)/len(X)

def std_dev(X):
    X_bar = mean(X)
    return np.sqrt(np.sum((X-X_bar)**2, axis=0))

def corr(X, Y):
    X_bar = mean(X)
    Y_bar = mean(Y)
    return (np.sum((X-X_bar)*(Y-Y_bar), axis=0)/(std_dev(X)*std_dev(Y)))

In [ ]:
trace0 = np.array(trace0)
trace1 = np.array(trace1)
trace2 = np.array(trace2)
trace3 = np.array(trace3)

trace_mix = np.array(trace_mix)

n_trace = n_iter
n_train = n_trace >> 1
n_trace_half = n_trace >> 1

train_trace = []

trace_end = 3450
trace_end_l = 13000

train_0 = trace0[0:n_train,75:trace_end].copy()
train_1 = trace1[0:n_train,75:trace_end].copy()
train_2 = trace2[0:n_train,75:trace_end].copy()
train_3 = trace3[0:n_train,75:trace_end].copy()
train_mix = trace_mix[0:n_train,100:trace_end_l].copy()

train_all = np.vstack((train_0,train_1,train_2,train_3))

test_trace0 = trace0[n_trace_half:n_trace,75:trace_end].copy()
test_trace1 = trace1[n_trace_half:n_trace,75:trace_end].copy()
test_trace_mix = trace_mix[n_trace_half:n_trace,100:trace_end_l].copy()

test_all_avg = np.mean(np.array(test_trace0), axis=0)
test_mix_avg = np.mean(np.array(test_trace_mix), axis=0)

In [ ]:
#select POIs by variance
train_cwnano_var = []
poi_s = []

n_poi = 1
interval = 500

trace_len = len(train_0[0])
pieces = int(trace_len/interval)

for i in range(trace_len):
    train_cwnano_var.append(np.var(train_all[:,i]))

for i in range(pieces):
    poi_nano_idx = np.array(train_cwnano_var[i*interval:i*interval+interval]).argsort()[-n_poi:][::-1]

    for idx in poi_nano_idx:
        true_idx = i*interval+idx
        poi_s.append(true_idx)

In [ ]:
# select POIs in the long trace
poi_m0 = []
poi_m1 = []
poi_m2 = []
poi_m3 = []
poi_st = []
cnt = 0
cnt_r = 0

for poi in tqdm(poi_s):

    v_points_0 = train_0[:,poi].copy()
    v_points_1 = train_1[:,poi].copy()
    v_points_2 = train_2[:,poi].copy()
    v_points_3 = train_3[:,poi].copy()

    features0 = []
    features1 = []
    features2 = []
    features3 = []

    for i in range(len(train_mix[0])):
         
        v_points_mix = train_mix[:,i].copy()
        cor0 = corr(v_points_0,v_points_mix)
        cor1 = corr(v_points_1,v_points_mix)
        cor2 = corr(v_points_2,v_points_mix)
        cor3 = corr(v_points_3,v_points_mix)
        features0.append(cor0)
        features1.append(cor1)
        features2.append(cor2)
        features3.append(cor3)

    if (max(features0) > 0.5 and max(features1)>0.5 and max(features2)>0.5 and max(features3)>0.5):
        id0 = np.argmax(features0)
        id1 = np.argmax(features1)
        id2 = np.argmax(features2)
        id3 = np.argmax(features3)

        poi_m0.append(id0)
        poi_m1.append(id1)
        poi_m2.append(id2)
        poi_m3.append(id3)
        
        poi_st.append(poi)



In [ ]:
# attack on the test set
iter_n = n_trace >> 1
cnt = 0
sig = 0


for i in trange(iter_n):
    
    # victim trace
    v_trace = test_trace_mix[i].copy() - test_mix_avg
    # template trace
    t_trace = test_trace0[i].copy() - test_all_avg

    trace_compact0 = np.array([v_trace[idx] for idx in poi_m0])
    trace_compact1 = np.array([v_trace[idx] for idx in poi_m1])
    trace_compact2 = np.array([v_trace[idx] for idx in poi_m2])
    trace_compact3 = np.array([v_trace[idx] for idx in poi_m3])

    trace_ref_compact = np.array([t_trace[idx] for idx in poi_st])
    
    c = np.zeros(4)

    c[0] = corr(trace_compact0,trace_ref_compact)
    c[1] = corr(trace_compact1,trace_ref_compact)
    c[2] = corr(trace_compact2,trace_ref_compact)
    c[3] = corr(trace_compact3,trace_ref_compact)
        
    if c.argmax(axis=0) == 0:
        cnt += 1
        
    sig += c[0] - max(c[1] , c[2] , c[3])
        
print("accuracy:\t", cnt/iter_n)     
print("significance:\t", sig/iter_n)

In [ ]:
scope.dis()